In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
initial_data = pd.read_csv('./data/algebra_2005_2006_train.txt', delimiter = "\t")
print(initial_data.shape)
initial_data.head()

(809694, 19)


,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,1,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,3(x+2) = 15,2005-09-09 12:24:35.0,2005-09-09 12:24:49.0,2005-09-09 12:25:15.0,2005-09-09 12:25:15.0,40.0,NaN,40.0,0,2,3,1,[SkillRule: Eliminate Parens; {CLT nested; CLT...,1
1,2,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,x+2 = 5,2005-09-09 12:25:15.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,16.0,16.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",1~~1
2,3,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,2-8y = -4,2005-09-09 12:25:36.0,2005-09-09 12:25:43.0,2005-09-09 12:26:12.0,2005-09-09 12:26:12.0,36.0,NaN,36.0,0,2,3,1,"[SkillRule: Remove constant; {ax+b=c, positive...",2
3,4,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,-8y = -6,2005-09-09 12:26:12.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,22.0,22.0,NaN,1,0,0,1,"[SkillRule: Remove coefficient; {ax+b=c, divid...",1~~1
4,5,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y-5 = -4,2005-09-09 12:26:38.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,118.0,118.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",3~~1


In [3]:
initial_data.describe()

,Row,Problem View,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects
count,8.096940e+05,809694.000000,808775.000000,620129.000000,188646.000000,809694.000000,809694.000000,809694.000000,809694.000000
mean,5.403042e+05,1.692636,27.744935,18.071478,59.544133,0.766514,0.504276,0.302483,1.174170
std,3.116039e+05,1.834555,55.246830,34.796694,88.241827,0.423049,2.036774,1.242620,0.804606
min,1.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.707982e+05,1.000000,5.000000,5.000000,16.000000,1.000000,0.000000,0.000000,1.000000
50%,5.394415e+05,1.000000,11.000000,8.000000,31.000000,1.000000,0.000000,0.000000,1.000000
75%,8.094898e+05,2.000000,27.000000,18.000000,66.000000,1.000000,0.000000,0.000000,1.000000
max,1.080616e+06,49.000000,2701.000000,1907.000000,2701.000000,1.000000,333.000000,217.000000,73.000000


# Collecter les données

In [4]:
initial_data.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'],
      dtype='object')

In [5]:
len(initial_data['Problem Hierarchy'].unique())

138

In [6]:
initial_data = initial_data.rename(columns={'Anon Student Id': 'user_id','Problem Name': 'item_id','Step Name': 'step_id','KC(Default)': 'kc_id','First Transaction Time': 'timestamp','Correct First Attempt': 'correct',"Hints":'nb_hint','Incorrects':'incorrect','Corrects':'incorrect_?'})
needed = initial_data[["user_id","item_id","timestamp","correct","nb_hint"]]
needed

,user_id,item_id,timestamp,correct,nb_hint
0,0BrbPbwCMz,EG4-FIXED,2005-09-09 12:24:49.0,0,3
1,0BrbPbwCMz,EG4-FIXED,2005-09-09 12:25:31.0,1,0
2,0BrbPbwCMz,EG40,2005-09-09 12:25:43.0,0,3
3,0BrbPbwCMz,EG40,2005-09-09 12:26:34.0,1,0
4,0BrbPbwCMz,EG40,2005-09-09 12:28:36.0,1,0
...,...,...,...,...,...
809689,IQQo3367x0,EG40,2006-03-09 10:58:13.0,1,0
809690,IQQo3367x0,EG40,2006-03-09 10:58:18.0,1,0
809691,IQQo3367x0,EG40,2006-03-09 10:58:22.0,0,3
809692,IQQo3367x0,EG40,2006-03-09 10:58:44.0,1,0


In [7]:
needed['timestamp'] = pd.to_datetime(needed['timestamp']).map(lambda t: t.timestamp()).round().astype(np.int32)

# Conservez uniquement les utilisateurs avec au moins 10 interactions
needed = needed.groupby("user_id").filter(lambda x: len(x) >= 10)

# Supprimer les entrées où le résultat n'est pas binaire mais continu
needed = needed[needed.correct.isin([0, 1])]
needed['correct'] = needed['correct'].astype(np.int32)

# Trier la colonne "timestamp" et changer le nom de la colonne
needed = needed.sort_values('timestamp')
print(needed.shape)
needed.head()

(809660, 5)


,user_id,item_id,timestamp,correct,nb_hint
535958,45TTYcotWk,LDEMO_WKST,1125395408,0,0
562642,HvghFVBS5v,LDEMO_WKST,1125395410,1,0
569598,IQ98biy227,LDEMO_WKST,1125395414,1,0
555691,Dmq6441349,LDEMO_WKST,1125395416,1,0
545705,80nlN05JQ6,LDEMO_WKST,1125395419,0,0


# Encodage

In [8]:
userId_encoder = LabelEncoder()
itemId_encoder = LabelEncoder()

needed["user_id"] = userId_encoder.fit_transform(needed["user_id"])
needed["item_id"] = itemId_encoder.fit_transform(needed["item_id"])

# Save user_id and item_id encoder
####
outputUserInEncoder = open('userId_encoder.pkl', 'wb')
pickle.dump(userId_encoder, outputUserInEncoder)
outputUserInEncoder.close()
####
outputItemInEncoder = open('itemId_encoder.pkl', 'wb')
pickle.dump(itemId_encoder, outputItemInEncoder)
outputItemInEncoder.close()

needed.head()

### Load saved encoder
"""
import pickle 
pkl_file = open('Departure_encoder.pkl', 'rb')
le_departure = pickle.load(pkl_file) 
pkl_file.close()
df_test['Departure'] = le_departure.transform(df_test['Departure'])
"""

"\nimport pickle \npkl_file = open('Departure_encoder.pkl', 'rb')\nle_departure = pickle.load(pkl_file) \npkl_file.close()\ndf_test['Departure'] = le_departure.transform(df_test['Departure'])\n"

In [9]:
# replace user_id and item_id where id == 0
needed["user_id"].replace(0,len(needed['user_id'].unique()),inplace=True)
needed["item_id"].replace(0,len(needed['item_id'].unique()),inplace=True)

In [10]:
conditions = [
    (needed['correct'] == 1) & (needed['nb_hint'] > 0),
    (needed['correct'] == 1) & (needed['nb_hint'] == 0),
    (needed['correct'] == 0) & (needed['nb_hint'] > 0),
    (needed['correct'] == 0) & (needed['nb_hint'] == 0)
    ]
# 1 ==> Correct avec hint
# 2 ==> Correct sans hint
# 3 ==> Incorrect avec hint
# 4 ==> Incorrect sans hint
values = [1, 2, 3, 4]

# create a new column and use np.select to assign values to it using our lists as arguments
needed['answers_using_hint'] = np.select(conditions, values)
needed.head()

,user_id,item_id,timestamp,correct,nb_hint,answers_using_hint
535958,72,563,1125395408,0,0,4
562642,247,563,1125395410,1,0,2
569598,249,563,1125395414,1,0,2
555691,212,563,1125395416,1,0,2
545705,154,563,1125395419,0,0,4


In [11]:
## dataConfig 
users = len(needed['user_id'].unique())
items = len(needed['item_id'].unique())
n_observations = len(needed)
config = {
    'user_ids': users,
    'item_ids': items,
    'observations': n_observations,
}
config

{'user_ids': 569, 'item_ids': 1084, 'observations': 809660}

In [39]:
# Save needed Data to csv file
# needed.to_csv("./data/neededData.csv")

In [12]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import table # EDIT: see deprecation warnings below

In [13]:
from bokeh.io import export_png, export_svgs
from bokeh.models import ColumnDataSource, DataTable, TableColumn

def save_df_as_image(df, path):
    source = ColumnDataSource(df)
    df_columns = [df.index.name]
    df_columns.extend(df.columns.values)
    columns_for_table=[]
    for column in df_columns:
        columns_for_table.append(TableColumn(field=column, title=column))

    data_table = DataTable(source=source, columns=columns_for_table,height_policy="auto",width_policy="auto",index_position=None)
    export_png(data_table, filename = path)

In [22]:
needed2 = needed[['user_id','item_id','correct','nb_hint','answers_using_hint']]
needed2

,user_id,item_id,correct,nb_hint,answers_using_hint
535958,72,563,0,0,4
562642,247,563,1,0,2
569598,249,563,1,0,2
555691,212,563,1,0,2
545705,154,563,0,0,4
...,...,...,...,...,...
478819,392,482,1,0,2
478820,392,482,0,4,3
478821,392,482,1,0,2
478822,392,482,1,0,2


In [21]:
exemple = needed.iloc[0:100,:]
exemple = exemple.reset_index()
exemple = exemple.drop('index', 1)
exemple = exemple[['user_id','item_id','correct','nb_hint','answers_using_hint']]
exemple

,user_id,item_id,correct,nb_hint,answers_using_hint
0,72,563,0,0,4
1,247,563,1,0,2
2,249,563,1,0,2
3,212,563,1,0,2
4,154,563,0,0,4
...,...,...,...,...,...
95,249,563,1,0,2
96,60,563,1,0,2
97,249,563,1,0,2
98,18,563,1,0,2


In [18]:
exemple

,user_id,item_id,correct,nb_hint,answers_using_hint,timestamp
0,72,563,0,0,4,1125395408
1,247,563,1,0,2,1125395410
2,249,563,1,0,2,1125395414
3,212,563,1,0,2,1125395416
4,154,563,0,0,4,1125395419
...,...,...,...,...,...,...
95,249,563,1,0,2,1125396088
96,60,563,1,0,2,1125396093
97,249,563,1,0,2,1125396094
98,18,563,1,0,2,1125396097


In [19]:
save_df_as_image(exemple, "plot.png")

In [20]:
save_df_as_image(initial_data, "initial_dataset.png")